### Project Description and Scope: 

You are provided with a collection of images of pets, that is, cats and dogs. These images are of different sizes with varied lighting conditions and they should be used as inputs for your model.
You are expected to write the code for CNN image classification model using TensorFlow that trains on the data and calculates the accuracy score on the test data. 

In [1]:
import tensorflow as tf

In [2]:
file_url = 'https://github.com/lukeNonyane1/Oreilly-Learning/raw/main/deep-learning-with-keras-tensorflow-workshop/ch3-image-classification-with-CNNs/1577957291_deeplearningwithkerasandtensorflow.zip'

In [3]:
# Download the dataset
zip_dir = tf.keras.utils.get_file('pet_classification', origin=file_url, extract=True)

In [4]:
zip_dir

'/Users/LNonyane/.keras/datasets/pet_classification'

In [5]:
# Import the pathlib library
import pathlib

In [6]:
# Create a variable called path containing the full path to the data directory using pathlib.Path(zip_dir).parent
path = pathlib.Path(zip_dir).parent / 'data'

In [7]:
print(path.parent)

/Users/LNonyane/.keras/datasets


In [8]:
# iterate and print each directory 
[x for x in path.iterdir() if x.is_dir()]

[PosixPath('/Users/LNonyane/.keras/datasets/data/test'),
 PosixPath('/Users/LNonyane/.keras/datasets/data/train')]

In [9]:
# Create two variables called train_dir and val_dir that take the full paths to the train and validation folders, respectively
train_dir = path / 'train'
val_dir = path / 'test'

In [10]:
train_cats_dir = train_dir / 'cats'
train_dogs_dir = train_dir / 'dogs'
val_cats_dir = val_dir / 'cats'
val_dogs_dir = val_dir / 'dogs'

In [11]:
import os

In [12]:
total_train = len(os.listdir(train_cats_dir)) + len(os.listdir(train_dogs_dir))
total_val = len(os.listdir(val_cats_dir)) + len(os.listdir(val_dogs_dir))
print('total train:',total_train)
print('total validation:',total_val)

total train: 40
total validation: 20


In [13]:
# import ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [14]:
# train_img_gen rescaled
train_img_gen = ImageDataGenerator(
    rescale=1./255
)

In [15]:
val_img_gen = ImageDataGenerator(rescale=1./255)

In [16]:
# batch_size, img_heght, img_width, channel
batch_size, img_heght, img_width, channel = 4, 100, 100, 3

In [17]:
# create a data generator called train_data_gen using flow_from_directory
train_data_gen = train_img_gen.flow_from_directory(batch_size=batch_size,
                                                  directory=train_dir,
                                                  target_size=(img_heght, img_width))

Found 40 images belonging to 2 classes.


In [18]:
# create a data generator called val_data_gen using flow_from_directory
val_data_gen = val_img_gen.flow_from_directory(batch_size=batch_size,
                                                  directory=val_dir,
                                                  target_size=(img_heght, img_width))

Found 20 images belonging to 2 classes.


In [19]:
# import required libraries
import numpy as np
from tensorflow.keras import layers

In [20]:
# set 8 as the seed for numpy and tensorflow using np.random.seed(8) and tf.random.set_seed(8); these values are arbitrary.
np.random.seed(8)
tf.random.set_seed(8)

In [21]:
model = tf.keras.Sequential([
    layers.Conv2D(32, 5, activation='relu', input_shape=(img_heght, img_width, 3)),
    layers.MaxPool2D(pool_size=(2,2), strides=1),
    layers.Conv2D(64, 5, activation='relu'),
    layers.MaxPool2D(pool_size=(2,2), strides=1),
    layers.Flatten(),
    layers.Dense(32, activation='relu'),
    layers.Dropout(rate=0.4),
    layers.Softmax(),
    layers.Dense(1, activation='sigmoid')
])

2022-05-31 07:03:42.702007: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [22]:
optimizer= tf.keras.optimizers.Adam(0.001)

In [23]:
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 96, 96, 32)        2432      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 95, 95, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 91, 91, 64)        51264     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 90, 90, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 518400)            0         
                                                                 
 dense (Dense)               (None, 32)                1

#### Fit the neural networks with fit() and provide the train and validation data generators, epochs=100, the steps per epoch, and the validation steps

In [25]:
model.fit(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=5,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size
)

Epoch 1/5
10/10 [==============================] - 2s 180ms/step - loss: 0.6976 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 2/5
10/10 [==============================] - 2s 167ms/step - loss: 0.6944 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 3/5
10/10 [==============================] - 2s 163ms/step - loss: 0.6950 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 4/5
10/10 [==============================] - 2s 164ms/step - loss: 0.6935 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 5/5
10/10 [==============================] - 2s 161ms/step - loss: 0.6937 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.5000


#### Improving model by leveraging pre-trained CNN

In [26]:
# import VGG16
from tensorflow.keras.applications import VGG16

In [27]:
# base_model
base_model = VGG16(input_shape=(img_heght,
                                img_width,
                                channel),
                   weights='imagenet',
                   include_top=False)

# freeze model so that weights will not be updated
base_model.trainable = False

In [28]:
# base_model summary
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 100, 100, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 100, 100, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 50, 50, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 50, 50, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 50, 50, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 25, 25, 128)       0     

In [29]:
# add fully connected layers to base_model
new_model = tf.keras.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(120, activation='relu'),
    layers.Dense(2, activation='sigmoid')
])

In [30]:
# Only the prediction_layers' weights will be updated when the model is trained
optimizer = tf.keras.optimizers.Adam(0.001)
new_model.compile(loss='binary_crossentropy',
                 optimizer=optimizer,
                 metrics=['accuracy'])

In [31]:
# summary of new model
new_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 3, 3, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 4608)              0         
                                                                 
 dense_2 (Dense)             (None, 120)               553080    
                                                                 
 dense_3 (Dense)             (None, 2)                 242       
                                                                 
Total params: 15,268,010
Trainable params: 553,322
Non-trainable params: 14,714,688
_________________________________________________________________


In [32]:
# fit model and provide the train and validation data generators, epochs=5, and the validation steps
new_model.fit(
    train_data_gen,
    steps_per_epoch=1,
    epochs=5,
    validation_data=val_data_gen,
    validation_steps=1
)

Epoch 1/5
1/1 [==============================] - 1s 720ms/step - loss: 0.7027 - accuracy: 0.5000 - val_loss: 0.7607 - val_accuracy: 0.7500
Epoch 2/5
1/1 [==============================] - 0s 184ms/step - loss: 1.6411 - accuracy: 0.2500 - val_loss: 0.9991 - val_accuracy: 0.2500
Epoch 3/5
1/1 [==============================] - 0s 184ms/step - loss: 1.1886 - accuracy: 0.0000e+00 - val_loss: 0.6095 - val_accuracy: 0.7500
Epoch 4/5
1/1 [==============================] - 0s 181ms/step - loss: 0.5819 - accuracy: 0.7500 - val_loss: 1.1460 - val_accuracy: 0.5000
Epoch 5/5
1/1 [==============================] - 0s 176ms/step - loss: 0.5300 - accuracy: 0.7500 - val_loss: 1.2843 - val_accuracy: 0.5000
